In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

import $exclude.$                        , $ivy.$                            // for cleaner logs
//import $profile.`hadoop-2.6`

import $ivy.$                                   // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

import Resolvers._

In [2]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

import $ivy.$                           

import vegas._

import vegas.data.External._

In [3]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)
// resolvers += "Will's bintray" at "https://dl.bintray.com/willb/maven/"
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "Will Benton",
  "https://dl.bintray.com/willb/maven/",
  MavenPattern,
  true
)

In [4]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1` // "org.isarnproject" %% "isarn-sketches" % "0.0.2"
import org.isarnproject.sketches._

import $ivy.$                                            // "org.isarnproject" %% "isarn-sketches" % "0.0.2"

import org.isarnproject.sketches._

In [5]:
import $ivy.`com.redhat.et::silex:0.1.2` // "com.redhat.et" %% "silex" % "0.1.2"

import $ivy.$                            // "com.redhat.et" %% "silex" % "0.1.2"

In [6]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint

import com.redhat.et.silex.util.vectors.implicits._
import com.redhat.et.silex.cluster.ClusteringTreeModel
import ClusteringTreeModel._

import org.apache.spark.rdd.RDD

import org.apache.spark.mllib.tree.RandomForest

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}

import org.apache.spark.mllib.evaluation.MulticlassMetrics

import org.apache.spark.mllib.regression.LabeledPoint


import com.redhat.et.silex.util.vectors.implicits._

import com.redhat.et.silex.cluster.ClusteringTreeModel

import ClusteringTreeModel._

In [7]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://frclust:7077")
  .appName("notebook")
  .getOrCreate()

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@210f9d35

In [8]:
implicit class TDEnhance(td: TDigest) extends Serializable {
  def toXY: Vector[(Double, Double)] = {
    val q = (0.001 +: (0.05 to 0.95 by 0.05) :+ 0.999).toVector
    q.map(q => (td.cdfInverse(q), q))
  }
}

defined class TDEnhance

In [9]:
val gaussianData = Vector.fill(1000000) { scala.util.Random.nextGaussian() }
val gaussianRDD = spark.sparkContext.parallelize(gaussianData, 10)

gaussianData: Vector[Double] = Vector(
  -1.0593943398272403,
  2.873323109332603,
  -1.7492097275474714,
  1.907027198224942,
  -0.4358307905369243,
  0.8007196482882192,
  0.003111350180089359,
  -8.039703984896764E-4,
  -0.030500561606107562,
  -2.2250375292228832,
  -0.3727359823907052,
...
gaussianRDD: RDD[Double] = ParallelCollectionRDD[0] at parallelize at cmd8.sc:2

In [10]:
val gaussianSketch = gaussianRDD.aggregate(TDigest.empty())(
    (td, x) => td + x,
    (td1, td2) => td1 ++ td2
)

gaussianSketch: TDigest = TDigest(
  0.5,
  0,
  124,
  Map(
    -4.592405047355569 -> 1.0,
    -4.522053501909571 -> 1.0,
    -4.459907226116781 -> 1.3124956866286168,
    -4.447455630086177 -> 0.6250053680854548,
    -4.388118766446965 -> 2.1503184382410723,
    -4.357066569724282 -> 0.9121805070448559,
    -4.31630828377465 -> 0.7772154592955327,
...

In [11]:
Vegas("CDF")
    .withXY(gaussianSketch.toXY)
    .encodeX("x", Quant)
    .encodeY("y", Quant)
    .mark(Line)
    .configCell(width=600, height=300)
    .show

<iframe id="frame-vegas-5885983f-f278-46d9-b322-0cf11efdd731" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative"
 }
 },
 "description" : "CDF",
 "data" : {
 "values" : [
 {
 "x" : -3.0952531722648415,
 "y" : 0.001
 },
 {
 "x" : -1.648371138980538,
 "y" : 0.05
 },
 {
 "x" : -1.2838819156598664,
 "y" : 0.1
 },
 {
 "x" : -1.0394377392242629,
 "y" : 0.15000000000000002
 },
 {
 "x" : -0.8369929200332961,
 "y" : 0.2
 },
 {
 "x" : -0.6785123078121453,
 "y" : 0.25
 },
 {
 "x" : -0.521779717438956,
 "y" : 0.3
 },
 {
 "x" : -0.3866899955165667,
 "y" : 0.35
 },
 {
 "x" : -0.2577096569354833,
 "y" : 0.39999999999999997
 },
 {
 "x" : -0.12475884810164087,
 "y" : 0.44999999999999996
 },
 {
 "x" : -0.00185707864221335,
 "y" : 0.49999999999999994
 },
 {
 "x" : 0.12638668204421838,
 "y" : 0.5499999999999999
 },
 {
 "x" : 0.25306468172291724,
 "y" : 0.6
 },
 {
 "x" : 0.3817005515904529,
 "y" : 0.65
 },
 {
 "x" : 0.525942405301457,
 "y" : 0.7000000000000001
 },
 {
 "x" : 0.6761115202943448,
 "y" : 0.7500000000000001
 },
 {
 "x" : 0.8402553570536371,
 "y" : 0.8000000000000002
 },
 {
 "x" : 1.0302029837407918,
 "y" : 0.8500000000000002
 },
 {
 "x" : 1.2757407098363127,
 "y" : 0.9000000000000002
 },
 {
 "x" : 3.0867302813871342,
 "y" : 0.999
 }
 ]
 },
 "config" : {
 "cell" : {
 "width" : 600.0,
 "height" : 300.0
 }
 }
}
 }
 vg.embed("#vegas-5885983f-f278-46d9-b322-0cf11efdd731", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-5885983f-f278-46d9-b322-0cf11efdd731'></div>
 </body>
</html>">

In [12]:
def loadCSV(fname: String) = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .option("mode", "DROPMALFORMED")
  .load(fname)

defined function loadCSV

In [13]:
val trainCSV = loadCSV("/data/tox21/tox21_dense_train.csv")
val testCSV = loadCSV("/data/tox21/tox21_dense_test.csv")
val trainLabelsCSV = loadCSV("/data/tox21/tox21_labels_train.csv")
val testLabelsCSV = loadCSV("/data/tox21/tox21_labels_test.csv")

trainCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
testCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
trainLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]
testLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]

In [14]:
def tdSketchFV(fv: RDD[Vector[Double]]): Vector[TDigest] = {
    val tds = fv.aggregate(Array.empty[TDigest])(
        (tdv, xv) => {
            if (tdv.isEmpty) {
                Array.tabulate(xv.length) { j => TDigest.empty(maxDiscrete=50) + xv(j) }
            } else {
                for { j <- 0 until xv.length } { tdv(j) += xv(j) }
                tdv
            }
        },
        (tdv1, tdv2) => {
            if (tdv1.isEmpty) tdv2 else {
                for { j <- 0 until tdv1.length } { tdv1(j) ++= tdv2(j) }
                tdv1
            }
        }
    )
    tds.toVector
}

defined function tdSketchFV

In [15]:
def synthesize(tdVec: Vector[TDigest], n: Int, partitions: Int = 2) = {
    implicit class AddSampling(td: TDigest) {
        import org.isarnproject.collections.mixmaps.nearest.Cover
        def cdfDiscreteInverse(q: Double): Double = {
            td.clusters.mCover(q * td.clusters.sum).map(n => (n.data.key, n.data.value)) match {
                case Cover(_, Some((x, _))) => x
                case Cover(Some((x, _)), None) => x
            }
        }

        def sample: Double = {
            val clust = td.clusters
            td.nclusters match {
                case 0 => 0.0
                case 1 => clust(clust.keyMin.get)
                case n if (n <= td.maxDiscrete) => cdfDiscreteInverse(scala.util.Random.nextDouble)
                case _ => td.cdfInverse(scala.util.Random.nextDouble)
            }
        }
    }
    val tdVecBC = spark.sparkContext.broadcast(tdVec)
    spark.sparkContext.parallelize(1 to n, partitions).map { r => tdVecBC.value.map(_.sample) }
}

defined function synthesize

In [96]:
val trainFV = trainCSV.rdd.map(_.toSeq.toVector.drop(1).map(_.asInstanceOf[String].toDouble))
val fvSketches = tdSketchFV(trainFV)

trainFV: RDD[Vector[Double]] = MapPartitionsRDD[62850] at map at cmd95.sc:1
fvSketches: Vector[TDigest] = Vector(
  TDigest(
    0.5,
    50,
    75,
    Map(
      1.0 -> 9.0,
      1.333 -> 24.0,
      1.5 -> 45.0,
      1.600489455071117 -> 33.242848970251714,
      1.6919536888451168 -> 64.71203977719783,
      1.728204980220985 -> 7.045111252550461,
...

In [97]:
val synthFV = synthesize(fvSketches, 48000, 4)

synthFV: RDD[Vector[Double]] = MapPartitionsRDD[62852] at map at cmd14.sc:22

In [98]:
val trainLab = trainFV.map(_.toLabeledPoint(1.0))
val synthLab = synthFV.map(_.toLabeledPoint(0.0))
val trainFR = (trainLab ++ synthLab).cache()

trainLab: RDD[LabeledPoint] = MapPartitionsRDD[62853] at map at cmd97.sc:1
synthLab: RDD[LabeledPoint] = MapPartitionsRDD[62854] at map at cmd97.sc:2
trainFR: RDD[LabeledPoint] = UnionRDD[62855] at $plus$plus at cmd97.sc:3

In [99]:
def frClassifier = {
val rfFR = RandomForest.trainClassifier(
    trainFR,              // training data
    2,                    // number of classes
    Map.empty[Int, Int],  // category info
    100,                  // forest size
    "auto",               // 
    "gini",               // split quality measure
    10,                   // max depth
    20,                   // max bins
    235711)               // random seed
val predictionAndLabels = trainLab.map { case LabeledPoint(label, features) =>
  val prediction = rfFR.predict(features)
  (prediction, label)
}
val metrics = new MulticlassMetrics(predictionAndLabels)
(rfFR, metrics)
}

defined function frClassifier

In [100]:
val (rfFR, metrics) = frClassifier
metrics.accuracy

rfFR: org.apache.spark.mllib.tree.model.RandomForestModel = TreeEnsembleModel classifier with 100 trees

metrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@13af5e45
res99_1: Double = 1.0

In [101]:
val frRules = rfFR.trees.map(_.rules(trainCSV.columns.toVector.drop(1), Map.empty[Int, Int]))

frRules: Array[Map[Double, Seq[Seq[Predicate]]]] = Array(
  Map(
    1.0 -> ArrayBuffer(
      List((MRVSA2 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 > 0.0), (PEOEVSA12 <= 0.0)),
      List(
        (MRVSA2 > 0.0),
        (RDFM30 > 0.0),
        (PEOEVSA4 > 0.0),
...

In [102]:
val featPerTree = for {
    rmap <- frRules
    feats = {
        val fraw = for {
            vrules <- rmap.values
            plist <- vrules
            pred <- plist
        } yield { pred.feature }
        fraw.toSet.toSeq
    }
    f <- feats
} yield { f }
val featCounts = featPerTree.foldLeft(Map.empty[String, Int])((m, f) => m + (f -> (1 + m.getOrElse(f, 0))))
val featHist = featCounts.toSeq.sortBy { case (_, n) => -n}.toVector
println(featHist.mkString("\n"))

(MRVSA7,20)
(RNCS,19)
(VSAEstate0,18)
(VSAEstate2,17)
(slogPVSA8,17)
(VSAEstate3,16)
(slogPVSA6,13)
(slogPVSA3,12)
(RDFM30,12)
(PEOEVSA4,12)
(RDFM29,11)
(RDFV30,10)
(RDFE30,10)
(slogPVSA0,9)
(PEOEVSA3,9)
(MRVSA8,8)
(Chi4c,8)
(PEOEVSA11,8)
(RDFU27,8)
(PEOEVSA10,8)
(slogPVSA7,7)
(MRVSA3,7)
(PEOEVSA12,6)
(PEOEVSA2,6)
(RDFV23,6)
(RDFV27,6)
(MRVSA2,6)
(RDFU30,6)
(RDFV29,6)
(RDFM27,6)
(RDFE28,6)
(RDFP30,6)
(RDFE29,6)
(Chi5ch,5)
(slogPVSA9,5)
(slogPVSA11,5)
(Chiv4c,5)
(RDFV24,5)
(Chiv5ch,5)
(RDFU25,5)
(RDFM25,4)
(RDFE26,4)
(RDFP29,4)
(EstateVSA6,4)
(RDFE27,4)
(RDFP26,4)
(PEOEVSA8,4)
(PEOEVSA13,4)
(RDFU29,4)
(MRVSA5,3)
(RDFP27,3)
(GeDi,3)
(RDFM22,3)
(RDFP21,3)
(RDFP25,3)
(EstateVSA2,3)
(RDFV25,3)
(slogPVSA2,3)
(RDFV26,3)
(RDFM24,3)
(RDFE25,3)
(Chi6ch,3)
(L1p,2)
(RDFP24,2)
(RDFU28,2)
(GATSm8,2)
(Chiv6ch,2)
(RDFM26,2)
(SPAN,2)
(RDFU24,2)
(RDFM19,2)
(bcutv2,2)
(EstateVSA1,2)
(rygr,2)
(L1u,2)
(PEOEVSA1,2)
(RDFV28,2)
(RDFU26,2)
(RDFM23,2)
(Tv,2)
(RDFP28,2)
(RDFM20,2)
(RDFE21,2)
(EstateVSA9,2)
(nta,

featPerTree: Array[String] = Array(
  "PEOEVSA12",
  "RDFM29",
  "RDFU28",
  "UI",
  "Chi4c",
  "RDFM30",
  "MRVSA2",
  "PEOEVSA4",
  "MoRSEP7",
  "naccr",
  "VSAEstate2",
...
featCounts: Map[String, Int] = Map(
  "MRVSA8" -> 8,
  "PEOEVSA12" -> 6,
  "nta" -> 1,
  "VSAEstate2" -> 17,
  "MRVSA5" -> 3,
  "Chi5ch" -> 5,
  "RDFM29" -> 11,
  "RDFP27" -> 3,
  "GeDi" -> 3,
  "L1p" -> 2,
  "MoRSEC21" -> 1,
...
featHist: Vector[(String, Int)] = Vector(
  ("MRVSA7", 20),
  ("RNCS", 19),
  ("VSAEstate0", 18),
  ("VSAEstate2", 17),
  ("slogPVSA8", 17),
  ("VSAEstate3", 16),
  ("slogPVSA6", 13),
  ("slogPVSA3", 12),
  ("RDFM30", 12),
  ("PEOEVSA4", 12),
  ("RDFM29", 11),
...

In [103]:
val featSelect = featHist.filter { case (_, n) => n > 1 }.map { case (f, _) => f }
featSelect.length
featHist.length

featSelect: Vector[String] = Vector(
  "MRVSA7",
  "RNCS",
  "VSAEstate0",
  "VSAEstate2",
  "slogPVSA8",
  "VSAEstate3",
  "slogPVSA6",
  "slogPVSA3",
  "RDFM30",
  "PEOEVSA4",
  "RDFM29",
...
res102_1: Int = 84
res102_2: Int = 143

In [104]:
val trainCSVFR = trainCSV.select("_c0", featSelect :_*)
val testCSVFR = testCSV.select("_c0", featSelect :_*)

trainCSVFR: DataFrame = [_c0: string, MRVSA7: string ... 83 more fields]
testCSVFR: DataFrame = [_c0: string, MRVSA7: string ... 83 more fields]

In [105]:
println(trainLabelsCSV.columns.mkString(", "))

_c0, NR.AhR, NR.AR, NR.AR.LBD, NR.Aromatase, NR.ER, NR.ER.LBD, NR.PPAR.gamma, SR.ARE, SR.ATAD5, SR.HSE, SR.MMP, SR.p53


In [106]:
def csvToPairs(csv: DataFrame) =
    csv.rdd.map(_.toSeq)
    .map(v => (v.head.asInstanceOf[String], v.tail.toVector.map(_.asInstanceOf[String].toDouble)))
def lpFR(tlab: String, labCSV: DataFrame, fvCSV: DataFrame) = {
    val labCol = labCSV.select("_c0", tlab)
    val labPairs = labCol.filter(labCol(tlab) =!= "NA")
        .rdd.map(_.toSeq).map { s => (s(0).asInstanceOf[String], s(1).asInstanceOf[String].toDouble) }
    val fvPairs = csvToPairs(fvCSV)
    val lp = labPairs.join(fvPairs).map { case (_, (lab, feats)) => LabeledPoint(lab, feats.toSpark) }
    lp
}
def trainAndTest(tlab: String) =
  (lpFR(tlab, trainLabelsCSV, trainCSVFR), lpFR(tlab, testLabelsCSV, testCSVFR))

defined function csvToPairs
defined function lpFR
defined function trainAndTest

In [107]:
def trainLRModel(train: RDD[LabeledPoint], test: RDD[LabeledPoint]) = {
    val lrModel = new LogisticRegressionWithLBFGS().run(train)
    val predictionAndLabels = test.map { case LabeledPoint(label, features) =>
        val prediction = lrModel.predict(features)
        (prediction, label)
    }
    val metrics = new MulticlassMetrics(predictionAndLabels)
    (lrModel, predictionAndLabels, metrics)
}

defined function trainLRModel

In [114]:
val allvsFR = trainLabelsCSV.columns.tail.map { acol => (acol, s"`$acol`")}
    .flatMap { case (acolRaw, acol) =>
        val (trainFR, testFR) = (lpFR(acol, trainLabelsCSV, trainCSVFR), lpFR(acol, testLabelsCSV, testCSVFR))
        val (trainAll, testAll) = (lpFR(acol, trainLabelsCSV, trainCSV), lpFR(acol, testLabelsCSV, testCSV))
        val (modelFR, predLabFR, metricsFR) = trainLRModel(trainFR, testFR)
        val (modelAll, predLabAll, metricsAll) = trainLRModel(trainAll, testAll)
        Seq(
            Map(
                "assay" -> acolRaw,
                "feats" -> modelAll.numFeatures,
                "accuracy" -> metricsAll.accuracy,
                "alt" -> metricsFR.accuracy
            ),
            Map(
                "assay" -> acolRaw,
                "feats" -> modelFR.numFeatures,
                "accuracy" -> metricsFR.accuracy,
                "alt" -> metricsAll.accuracy
            )
        )
    }

allvsFR: Array[Map[String, Any]] = Array(
  Map(
    "assay" -> NR.AhR,
    "feats" -> 801,
    "accuracy" -> 0.8754098360655738,
    "alt" -> 0.8721311475409836
  ),
  Map(
    "assay" -> NR.AhR,
    "feats" -> 84,
    "accuracy" -> 0.8721311475409836,
    "alt" -> 0.8754098360655738
...

In [125]:
Vegas().
  withData(allvsFR).
  mark(Bar).
  addTransformCalculation("features", """ datum.feats < 800 ? (datum.accuracy >= datum.alt ? "Reduced (84) >=" : "Reduced (84) <") : "Full (801)" """).
  encodeColumn("assay", Nominal, scale=Scale(padding=20.0),
               axis=Axis(title="tox assay", orient=Orient.Bottom, axisWidth=1.0, labelAngle=45,labelAlign="bottom",offset= -5)).
  encodeY("accuracy", Quantitative, axis=Axis(title="model accuracy", grid=false)).
  encodeX("feats", Nominal, scale=Scale(bandSize = 15), hideAxis=true).
  encodeColor("features", Nominal, scale=Scale(rangeNominals=List("#c25b3b","#6c86c2", "#1d28c2"))).
  configFacet(cell=CellConfig(strokeWidth = 0)).
  show

<iframe id="frame-vegas-18b065a1-9874-4100-900e-b449b0950f33" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "column" : {
 "axis" : {
 "labelAngle" : 45.0,
 "orient" : "bottom",
 "title" : "tox assay",
 "axisWidth" : 1.0,
 "offset" : -5.0,
 "labelAlign" : "bottom"
 },
 "scale" : {
 "padding" : 20.0
 },
 "field" : "assay",
 "type" : "nominal"
 },
 "x" : {
 "axis" : false,
 "scale" : {
 "bandSize" : 15.0
 },
 "field" : "feats",
 "type" : "nominal"
 },
 "y" : {
 "axis" : {
 "title" : "model accuracy",
 "grid" : false
 },
 "field" : "accuracy",
 "type" : "quantitative"
 },
 "color" : {
 "scale" : {
 "range" : [
 "#c25b3b",
 "#6c86c2",
 "#1d28c2"
 ]
 },
 "field" : "features",
 "type" : "nominal"
 }
 },
 "data" : {
 "values" : [
 {
 "assay" : "NR.AhR",
 "feats" : 801,
 "accuracy" : 0.8754098360655738,
 "alt" : 0.8721311475409836
 },
 {
 "assay" : "NR.AhR",
 "feats" : 84,
 "accuracy" : 0.8721311475409836,
 "alt" : 0.8754098360655738
 },
 {
 "assay" : "NR.AR",
 "feats" : 801,
 "accuracy" : 0.9744027303754266,
 "alt" : 0.9761092150170648
 },
 {
 "assay" : "NR.AR",
 "feats" : 84,
 "accuracy" : 0.9761092150170648,
 "alt" : 0.9744027303754266
 },
 {
 "assay" : "NR.AR.LBD",
 "feats" : 801,
 "accuracy" : 0.9725085910652921,
 "alt" : 0.9828178694158075
 },
 {
 "assay" : "NR.AR.LBD",
 "feats" : 84,
 "accuracy" : 0.9828178694158075,
 "alt" : 0.9725085910652921
 },
 {
 "assay" : "NR.Aromatase",
 "feats" : 801,
 "accuracy" : 0.9185606060606061,
 "alt" : 0.9223484848484849
 },
 {
 "assay" : "NR.Aromatase",
 "feats" : 84,
 "accuracy" : 0.9223484848484849,
 "alt" : 0.9185606060606061
 },
 {
 "assay" : "NR.ER",
 "feats" : 801,
 "accuracy" : 0.9050387596899225,
 "alt" : 0.9069767441860465
 },
 {
 "assay" : "NR.ER",
 "feats" : 84,
 "accuracy" : 0.9069767441860465,
 "alt" : 0.9050387596899225
 },
 {
 "assay" : "NR.ER.LBD",
 "feats" : 801,
 "accuracy" : 0.9616666666666667,
 "alt" : 0.9616666666666667
 },
 {
 "assay" : "NR.ER.LBD",
 "feats" : 84,
 "accuracy" : 0.9616666666666667,
 "alt" : 0.9616666666666667
 },
 {
 "assay" : "NR.PPAR.gamma",
 "feats" : 801,
 "accuracy" : 0.9256198347107438,
 "alt" : 0.943801652892562
 },
 {
 "assay" : "NR.PPAR.gamma",
 "feats" : 84,
 "accuracy" : 0.943801652892562,
 "alt" : 0.9256198347107438
 },
 {
 "assay" : "SR.ARE",
 "feats" : 801,
 "accuracy" : 0.8162162162162162,
 "alt" : 0.8198198198198198
 },
 {
 "assay" : "SR.ARE",
 "feats" : 84,
 "accuracy" : 0.8198198198198198,
 "alt" : 0.8162162162162162
 },
 {
 "assay" : "SR.ATAD5",
 "feats" : 801,
 "accuracy" : 0.9421221864951769,
 "alt" : 0.9356913183279743
 },
 {
 "assay" : "SR.ATAD5",
 "feats" : 84,
 "accuracy" : 0.9356913183279743,
 "alt" : 0.9421221864951769
 },
 {
 "assay" : "SR.HSE",
 "feats" : 801,
 "accuracy" : 0.9606557377049181,
 "alt" : 0.9557377049180328
 },
 {
 "assay" : "SR.HSE",
 "feats" : 84,
 "accuracy" : 0.9557377049180328,
 "alt" : 0.9606557377049181
 },
 {
 "assay" : "SR.MMP",
 "feats" : 801,
 "accuracy" : 0.8931860036832413,
 "alt" : 0.8747697974217311
 },
 {
 "assay" : "SR.MMP",
 "feats" : 84,
 "accuracy" : 0.8747697974217311,
 "alt" : 0.8931860036832413
 },
 {
 "assay" : "SR.p53",
 "feats" : 801,
 "accuracy" : 0.922077922077922,
 "alt" : 0.9237012987012987
 },
 {
 "assay" : "SR.p53",
 "feats" : 84,
 "accuracy" : 0.9237012987012987,
 "alt" : 0.922077922077922
 }
 ]
 },
 "transform" : {
 "calculate" : [
 {
 "field" : "features",
 "expr" : " datum.feats < 80

In [126]:
testCSV.count

res125: Long = 647L

In [128]:
testLabelsCSV.select("_c0", "`NR.AhR`", "`NR.AR`", "`NR.AR.LBD`", "`NR.Aromatase`").show

+---------------+------+-----+---------+------------+
|            _c0|NR.AhR|NR.AR|NR.AR.LBD|NR.Aromatase|
+---------------+------+-----+---------+------------+
|NCGC00261900-01|     0|    1|       NA|           0|
|NCGC00260869-01|     0|    1|       NA|          NA|
|NCGC00261776-01|     1|    1|        0|          NA|
|NCGC00261380-01|    NA|    0|       NA|           1|
|NCGC00261842-01|     0|    0|        0|          NA|
|NCGC00261662-01|     1|    0|        0|          NA|
|NCGC00261190-01|    NA|    0|        0|          NA|
|NCGC00260731-01|    NA|    0|        0|          NA|
|NCGC00261926-01|     0|    0|       NA|          NA|
|NCGC00261579-01|    NA|    0|       NA|           0|
|NCGC00260921-01|     0|    0|        0|           1|
|NCGC00261119-01|     1|    0|        0|           1|
|NCGC00260957-01|    NA|    0|       NA|          NA|
|NCGC00261812-01|    NA|    0|        0|           0|
|NCGC00260831-01|     1|    0|       NA|          NA|
|NCGC00261544-01|     0|    